In [ ]:
pip install torch torchvision

In [1]:
import torch
import cv2

In [2]:
import numpy as np
import pandas as pd
import os
from PIL import Image 
from torch.utils.data import Dataset 
from torchvision import datasets, transforms, models
import torchvision.transforms as transforms # A module from the torchvision library that provides common image transformations, such as resizing, cropping, and normalization.
from torch.utils.data import random_split # A function from PyTorch that allows for randomly splitting a dataset into training and validation subsets.
#from torch.optim.lr_scheduler import ReduceLROnPlateau # A PyTorch scheduler that adjusts the learning rate during training based on a specified metric, reducing it when the metric plateaus.
import torch.nn as nn # A module in PyTorch that provides classes for defining and building neural networks.
from torchvision import utils # A module from torchvision that contains utility functions for working with images, such as saving and visualizing them.
from torch.utils.data import Dataset
from PIL import Image
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torch.optim as optim
#import splitfolders

#import torch.nn.functional as F
import pathlib
from sklearn.metrics import confusion_matrix, classification_report
import itertools # This import statement imports the itertools module, which provides functions for efficient looping and combining of iterables. It can be used for tasks such as generating combinations or permutations of elements.
from tqdm.notebook import trange, tqdm # These functions allow for the creation of progress bars to track the progress of loops or tasks.
from torch import optim
import warnings
warnings.filterwarnings('ignore')


In [3]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [4]:
! pip install wget nibabel # lecture files format .nii



In [5]:
# ask user to do the commands in the cells in both researcher and node environments

!git clone https://github.com/ultralytics/yolov5  # clone

%cd yolov5
%pip install -qr requirements.txt comet_ml  # install

fatal: destination path 'yolov5' already exists and is not an empty directory.
/home/ybouilla/github/fedbiomed/notebooks/yolov5
Note: you may need to restart the kernel to use updated packages.


In [6]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

torch.save(model.state_dict(), 'mymodel.pt')

Using cache found in /user/ybouilla/home/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-1-19 Python-3.10.13 torch-1.13.1+cu117 CUDA:0 (NVIDIA RTX 2000 Ada Generation Laptop GPU, 7943MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [7]:
model.eval()

AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

In [8]:
from models.yolo import Model

architecture = Model(cfg='models/yolov5s.yaml')


                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  3    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]              
  8                -1  1   1182720  

In [ ]:
architecture.forward

In [ ]:
import torch.nn as nn

classifier_layer = nn.Sequential(nn.Conv3d(16, 2, kernel_size=(1, 1, 1), stride=(1, 1, 1)))

In [15]:
architecture.m

AttributeError: 'DetectionModel' object has no attribute 'm'

# Edit your training plan

In [ ]:
from fedbiomed.common.training_plans import TorchTrainingPlan
from flamby.datasets.fed_ixi import Baseline, BaselineLoss, Optimizer
from fedbiomed.common.data import FlambyDataset, DataManager
from models.yolo import Model

class MyTrainingPlan(TorchTrainingPlan):

    
    def init_model(self, model_args):
        return Model(cfg='models/yolov5s.yaml')

    def init_optimizer(self, optimizer_args):
        return Optimizer(self.model().parameters(), lr=optimizer_args["lr"])

    def init_dependencies(self):
        return ["from flamby.datasets.fed_ixi import Baseline, BaselineLoss, Optimizer",
                "from fedbiomed.common.data import FlambyDataset, DataManager",
               "from models.yolo import Model"]

    def training_step(self, data, target):
        output = self.model().forward(data)
        return BaselineLoss().forward(output, target)

    def training_data(self):
        dataset = FlambyDataset()
        loader_arguments = { 'shuffle': True}
        return DataManager(dataset, **loader_arguments)

In [16]:
from flamby.datasets.fed_ixi import Baseline, BaselineLoss


In [ ]:
tp = MyTrainingPlan()

tp.import_model('mymodel.pt')

In [ ]:


model_args = {}

training_args = {
    'loader_args': { 'batch_size': 8, },
    'optimizer_args': {
        "lr" : 1e-3
    },
    'epochs': 1,
    'dry_run': False,
    'batch_maxnum': 2 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}


In [ ]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['ixi']
rounds = 1

exp = Experiment(tags=tags,
                 model_args=model_args,
                 training_plan_class=MyTrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

In [ ]:
exp.training_plan().import_model('mymodel.pt')

In [ ]:
exp.run()

In [17]:
Baseline().classifier


ConvolutionalBlock(
  (conv_layer): Conv3d(16, 2, kernel_size=(1, 1, 1), stride=(1, 1, 1))
  (block): Sequential(
    (0): Conv3d(16, 2, kernel_size=(1, 1, 1), stride=(1, 1, 1))
  )
)

In [20]:
Baseline().classifier.conv_layer


Conv3d(16, 2, kernel_size=(1, 1, 1), stride=(1, 1, 1))